# Lux level control using DQN

### Install and import necessary libraries

In [1]:
# !pip install tensorflow==2.3.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

### Environment

In [2]:
class LightEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # lux level array
        self.observation_space = Box(low=np.array([0]), high=np.array([1000])) # real valued quantities
        # Set start level
        self.state = 600 + random.randint(-3,3)
        # Set illumination length
        self.light_length = 1000
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 lux level
        # 1 -1 = 0 
        # 2 -1 = 1 lux level 
        self.state += action -10 
        # Reduce illumination length by 1 second
        self.light_length -= 10 
        
        # Calculate reward
        if self.state >=550 and self.state <=650: 
            reward = 1 
        else: 
            reward = -1 
        
        # Check if illumination process is done
        if self.light_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply lux level noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset light lux level
        self.state = 600 + random.randint(-3,3)
        # Reset illumination time
        self.light_length = 1000
        return self.state

In [3]:
# initialize the environment
env = LightEnv()

c:\Users\h4rip\anaconda3\envs\tf\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:
# print a sample form the observation space
env.observation_space.sample()

array([624.5526], dtype=float32)

In [5]:
# Run the simulation for some iterations
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-90
Episode:2 Score:-90
Episode:3 Score:-90
Episode:4 Score:-90
Episode:5 Score:-90
Episode:6 Score:-90
Episode:7 Score:-90
Episode:8 Score:-90
Episode:9 Score:-90
Episode:10 Score:-90


### Deep Q Learning

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
states = env.observation_space.shape    # Shape of Input
actions = env.action_space.n            # Shape of output

In [8]:
actions

3

In [9]:
# Deep Neural Network for Q value approximation
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [12]:
# build a DQN agent
def build_agent(model, actions):
    policy = BoltzmannQPolicy() # Used for discrete action space
    memory = SequentialMemory(limit=50000, window_length=1) # used to srtore the samples for training
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [13]:
# Train the DQN agent
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

c:\Users\h4rip\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)


c:\Users\h4rip\anaconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 2:36:33 - reward: 1.0000

c:\Users\h4rip\anaconda3\envs\tf\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 67s 7ms/step - reward: -0.8998
100 episodes - episode_reward: -89.980 [-92.000, -88.000] - loss: 0.695 - mae: 40.129 - mean_q: -17.817

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -0.8988
100 episodes - episode_reward: -89.880 [-92.000, -88.000] - loss: 0.420 - mae: 21.143 - mean_q: -30.252

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -0.8982
100 episodes - episode_reward: -89.820 [-92.000, -88.000] - loss: 0.389 - mae: 21.650 - mean_q: -31.456

Interval 4 (30000 steps performed)
 3556/10000 [=========>....................] - ETA: 44s - reward: -0.8965done, took 229.414 seconds


In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)